## Markowitz portfolio design using [Pandas](http://pandas.pydata.org), [Bokeh](http://bokeh.pydata.org/en/latest/), & [Gurobi](http://www.gurobi.com)
### Michael C. Grant, Ph.D., [Continuum Analytics](http://continuum.io)

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import sys
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import Range1d, HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

Our sample data consists of normalized return data from the Dow Jones Industrial Average portfolio companies in 2012. The original data comes to us courtesy of [Ronald Hochreiter](http://www.hochreiter.net/ronald/index/)'s site [Finance-R](http://www.finance-r.com). We used [R](https://www.r-project.org/about.html) itself, along with the [rpy2](http://rpy.sourceforge.net) R/Python connector, to load this data and convert it to a Python-friendly form.

*My preprocessed CSV file is available for download [here](http://pastebin.com/BX9mBNhL). Make sure to rename it to 'markowitz.csv' after download!*

In [2]:
data = pd.DataFrame.from_csv('markowitz.csv')
N = data.shape[1]
data

,AA,AXP,BA,BAC,CAT,CSCO,CVX,DD,DIS,GE,...,MSFT,PFE,PG,T,TRV,UNH,UTX,VZ,WMT,XOM
2012-01-10,0.022099,0.009492,0.010469,0.142114,0.063579,0.010520,-0.011893,0.013590,0.034455,0.019886,...,0.040172,-0.001443,-0.007540,-0.006046,0.018762,0.021761,0.018354,-0.016739,-0.021324,-0.003262
2012-01-11,0.019438,0.014545,0.005436,0.181661,0.050463,0.003802,-0.021921,0.005824,-0.003921,0.017426,...,0.011814,0.005825,-0.016816,0.000000,0.019722,0.014821,0.021995,0.004925,-0.005039,-0.010992
2012-01-12,0.061069,0.017295,0.026909,0.076555,0.067208,0.011996,-0.037828,0.030002,-0.019537,0.020810,...,0.011694,0.018102,-0.010731,0.005330,0.008021,0.005230,0.039106,0.012397,0.001397,-0.011873
2012-01-13,0.070156,0.030756,0.008402,0.070033,0.070172,0.010947,-0.020555,0.051247,-0.037904,0.010067,...,0.005201,0.012249,-0.008227,0.012912,0.000349,-0.001544,0.029503,0.015193,0.009144,-0.002808
2012-01-17,0.035714,0.037831,0.009591,0.033708,0.064516,0.017945,-0.025321,0.045372,-0.031928,-0.006084,...,0.018818,0.005327,-0.005671,0.019742,0.000000,0.016232,0.039647,0.017113,0.011396,0.002187
2012-01-18,0.061622,0.038863,0.000829,0.025797,0.047678,0.037808,-0.020414,0.049162,-0.015373,0.016156,...,0.013873,0.001445,0.003482,0.019320,-0.003442,0.025169,0.020617,0.011532,0.016342,0.008485
2012-01-19,0.057203,0.040931,0.010951,0.013177,0.066093,0.037879,-0.007796,0.044543,0.019155,0.014365,...,0.014313,-0.002896,0.014226,0.014200,0.006562,-0.013067,0.006362,0.002723,0.020258,0.022960
2012-01-20,0.024666,0.007989,0.000137,0.041481,0.040969,0.040409,0.018207,0.027595,0.015207,0.011570,...,0.061148,-0.004325,0.014356,0.013079,0.029926,-0.011368,-0.007124,0.001361,0.025458,0.032369
2012-01-23,0.045786,-0.006336,0.012221,0.097412,0.042570,0.040606,0.008711,0.019587,0.022210,0.005537,...,0.052476,-0.005808,-0.004466,0.010977,0.015508,-0.005026,0.010371,-0.013333,0.023001,0.030481
2012-01-24,0.052247,-0.019644,0.001515,0.125776,0.032903,0.026175,0.000000,0.018012,0.020053,0.005008,...,0.038050,-0.013006,-0.018853,-0.005280,-0.028250,-0.041833,0.009567,-0.031479,0.025654,0.017340


We can look at this data in a variety of ways. First, some bulk statistics:

In [3]:
stats = pd.concat((data.mean(),data.std(),(data+1).prod()-1),axis=1)
stats.columns = ['Mean_return', 'Volatility', 'Total_return']
stats

,Mean_return,Volatility,Total_return
AA,-0.000587,0.038273,-0.275165
AXP,0.003952,0.026716,1.410810
BA,0.001162,0.023690,0.241494
BAC,0.014642,0.055879,23.445925
CAT,-0.000425,0.037938,-0.244639
CSCO,0.002091,0.038484,0.390132
CVX,0.000698,0.024279,0.104092
DD,0.000382,0.027596,-0.000811
DIS,0.005379,0.023158,2.487461
GE,0.003277,0.026448,1.046976


In [4]:
extremes = pd.concat((stats.idxmin(),stats.min(),stats.idxmax(),stats.max()),axis=1)
extremes.columns = ['Minimizer','Minimum','Maximizer','Maximum']
extremes

,Minimizer,Minimum,Maximizer,Maximum
Mean_return,HPQ,-0.011060,BAC,0.014642
Volatility,JNJ,0.015192,BAC,0.055879
Total_return,HPQ,-0.952484,BAC,23.445925


Now let's plot the growth of each stock over the course of the year, normalized so that each stock starts at 1.00 at the beginning of the year. Since the data was already given to us in units of incremental growth ($P_k/P_{k-1}-1$), we have to do a bit of backtracking to recover the original historical trends.

In [5]:
growth = (data+1.0).cumprod()
tx = growth.index
syms = growth.columns
esyms = pd.concat((extremes['Minimizer'],extremes['Maximizer']))

fig = figure(x_axis_type='datetime', y_axis_type='log', tools="pan,box_zoom,reset,resize")

# Plot the extreme stocks
for (symb,c) in zip(esyms,{'green','red','blue','purple'}):
    fig.line(tx, growth[symb], legend=symb, color=c, line_width=2)
    fig.text([tx[-1]], [growth[symb][-1]], [symb], text_font_size='10px', text_baseline='middle', alpha=1.0)
    
# Equal allocation
eq_growth = growth.mean(axis=1)
fig.line(tx, eq_growth, color='maroon', legend='equal allocation', line_width=2)

# All the other stocks, and the text labels
for symb in syms:
    if symb not in esyms:
        fig.line(tx, growth[symb], color='maroon', alpha=0.25, name=symb)
        fig.text([tx[-1]], [growth[symb][-1]], [symb], text_font_size='10px', text_baseline='middle', alpha=0.25)

# Expand the y-axis to give us a good view
fig.y_range = Range1d(growth.values.min()/1.25,1.25*growth.values.max())
fig.legend.location='bottom_left'
show(fig)

As we move towards our Markowitz portfolio designs it makes sense to view the stocks on a mean/variance scatter plot. We can clearly see that `BAC` and `HPQ` are performance outliers; sadly for `HPQ`, in opposite directions...

In [6]:
fig = figure(tools="pan,box_zoom,reset,resize")
source = ColumnDataSource(stats)
hover = HoverTool(tooltips=[('Symbol','@index'),('Volatility','@Volatility'),('Mean return','@Mean_return')])
fig.add_tools(hover)
fig.circle('Volatility', 'Mean_return', size=5, color='maroon', source=source)
fig.text('Volatility', 'Mean_return', syms, text_font_size='10px', x_offset=3, y_offset=-2, source=source)
fig.xaxis.axis_label='Volatility (standard deviation)'
fig.yaxis.axis_label='Mean return'
show(fig)

/Users/Ricardo/anaconda/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


### Gurobi

Time to bring in the big guns. Expressed in mathematical terms, we will be solving models in this form:

$$\begin{array}{lll}
\text{minimize}   & x^T \Sigma x \\
\text{subject to} & \sum_i x_i = 1 & \text{fixed budget} \\
                  & r^T x = \gamma & \text{fixed return} \\
                  & x \geq 0
\end{array}$$

In this model, the optimization variable $x\in\mathbb{R}^N$ is a vector representing the fraction of the budget allocated to each stock; that is, $x_i$ is the amount allocated to stock $i$. The paramters of the model are the mean returns $r$, a *covariance matrix* $\Sigma$, and the target return $\gamma$. What we will do is sweep $\gamma$ between the worst and best returns we have seen above, and compute the portfolio that achieves the target return but with as little risk as possible.

The *covariance matrix* $\Sigma$ merits some explanation. Along the diagonal, it contains the squares of the volatilities (standard deviations) computed above. But off the diagonal, it contains measures of the *correlation* between two stocks: that is, whether they tend to move in the same direction (positive correlation), in opposite directions (negative correlation), or a mixture of both (small correlation). This entire matrix is computed with a single call to Pandas:

In [7]:
Sigma = data.cov()
Sigma

,AA,AXP,BA,BAC,CAT,CSCO,CVX,DD,DIS,GE,...,MSFT,PFE,PG,T,TRV,UNH,UTX,VZ,WMT,XOM
AA,0.001465,0.000577,0.000449,0.001532,0.001131,0.000747,0.000507,0.000741,0.000378,0.000612,...,0.000541,0.000121,0.000149,0.000270,0.000472,0.000060,0.000588,0.000196,0.000126,0.000486
AXP,0.000577,0.000714,0.000321,0.000859,0.000481,0.000411,0.000304,0.000420,0.000274,0.000409,...,0.000375,0.000115,0.000091,0.000233,0.000257,0.000136,0.000364,0.000207,0.000066,0.000294
BA,0.000449,0.000321,0.000561,0.000449,0.000389,0.000346,0.000279,0.000357,0.000212,0.000310,...,0.000245,0.000162,0.000083,0.000118,0.000277,0.000027,0.000283,0.000116,0.000050,0.000263
BAC,0.001532,0.000859,0.000449,0.003122,0.001261,0.001059,0.000545,0.000820,0.000610,0.000763,...,0.000733,0.000143,0.000194,0.000233,0.000507,0.000225,0.000774,0.000179,0.000180,0.000557
CAT,0.001131,0.000481,0.000389,0.001261,0.001439,0.000637,0.000442,0.000696,0.000305,0.000558,...,0.000645,0.000104,0.000233,0.000132,0.000377,0.000155,0.000687,0.000083,0.000080,0.000477
CSCO,0.000747,0.000411,0.000346,0.001059,0.000637,0.001481,0.000407,0.000557,0.000220,0.000434,...,0.000480,0.000150,0.000172,0.000132,0.000340,0.000251,0.000572,0.000039,-0.000036,0.000389
CVX,0.000507,0.000304,0.000279,0.000545,0.000442,0.000407,0.000589,0.000391,0.000347,0.000431,...,0.000393,0.000216,0.000237,0.000286,0.000292,0.000157,0.000358,0.000251,0.000071,0.000415
DD,0.000741,0.000420,0.000357,0.000820,0.000696,0.000557,0.000391,0.000762,0.000328,0.000478,...,0.000469,0.000196,0.000108,0.000224,0.000308,0.000138,0.000429,0.000135,0.000034,0.000349
DIS,0.000378,0.000274,0.000212,0.000610,0.000305,0.000220,0.000347,0.000328,0.000536,0.000326,...,0.000342,0.000195,0.000163,0.000240,0.000249,0.000121,0.000283,0.000213,0.000126,0.000263
GE,0.000612,0.000409,0.000310,0.000763,0.000558,0.000434,0.000431,0.000478,0.000326,0.000699,...,0.000399,0.000227,0.000161,0.000271,0.000288,0.000168,0.000309,0.000254,0.000148,0.000412


In [8]:
Sigma.loc[['CSCO','WMT'],['CSCO','WMT']]

,CSCO,WMT
CSCO,0.001481,-0.000036
WMT,-0.000036,0.000587


### Building the base model

We are not solving just one model here, but literally hundreds of them, with different return targets and with the short constraints added or removed. One very nice feature of the Gurobi Python interface is that we can build a single "base" model, and reuse it for each of these scenarios by adding and removing constraints.

First, let's initialize the model and declare the variables. As we mentioned above, we're creating separate variables for the long and short positions. We put these variables into a Pandas DataFrame for easy organization, and create a third column that holds the difference between the long and short variables---that is, the net allocations for each stock. Another nice feature of Gurobi's Python interface is that the variable objects can be used in simple linear and quadratic expressions using familar Python syntax.

In [9]:
# Import the Gurobi python module
from gurobipy import *

# Instantiate our model
m = Model("portfolio")

# Create one variable for each stock
portvars = [m.addVar(name=symb,lb=0.0) for symb in syms]
portvars = pd.Series(portvars, index=syms)
portfolio = pd.DataFrame({'Variables':portvars})

# Commit the changes to the model
m.update()

portfolio

,Variables
AA,<gurobi.Var AA>
AXP,<gurobi.Var AXP>
BA,<gurobi.Var BA>
BAC,<gurobi.Var BAC>
CAT,<gurobi.Var CAT>
CSCO,<gurobi.Var CSCO>
CVX,<gurobi.Var CVX>
DD,<gurobi.Var DD>
DIS,<gurobi.Var DIS>
GE,<gurobi.Var GE>


Now let us construct some intermediate expressions representing quantities like the portfolio return, portfolio risk, total long budget, total short budget, and total budget. Again, thanks to Gurobi's convenient Python interface, we can use the *same Pandas calls* we would use with normal numeric quantities to operate on our optimization variables. As an example, we're printing the expression representing the portfolio return, which we constructed using a single Pandas call.

In [10]:
# The total budget
p_total = portvars.sum()

# The mean return for the portfolio
p_return = stats['Mean_return'].dot(portvars)

# The (squared) volatility of the portfolio
p_risk = Sigma.dot(portvars).dot(portvars)

In [11]:
p_total

<gurobi.LinExpr: AA + AXP + BA + BAC + CAT + CSCO + CVX + DD + DIS + GE + HD + HPQ + IBM + INTC + JNJ + JPM + KO + MCD + MMM + MRK + MSFT + PFE + PG + T + TRV + UNH + UTX + VZ + WMT + XOM>

In [12]:
p_return

<gurobi.LinExpr: -0.0005870566974 AA + 0.00395226831607 AXP + 0.00116202844324 BA + 0.0146422931372 BAC + -0.000424581435822 CAT + 0.00209132378788 CSCO + 0.000698317246613 CVX + 0.000382062402251 DD + 0.00537905996493 DIS + 0.00327688776275 GE + 0.00805819875154 HD + -0.0110603729521 HPQ + 0.00145327639213 IBM + -0.00303270221244 INTC + 0.00226496275548 JNJ + 0.00570128690755 JPM + 0.0017114443111 KO + -0.00156406839896 MCD + 0.0028190050214 MMM + 0.00243681865327 MRK + 0.000433781381147 MSFT + 0.00386527135517 PFE + 0.00125521692274 PG + 0.0034322064019 T + 0.00482319468265 TRV + 0.0014381115319 UNH + 0.00280306952359 UTX + 0.00328971247846 VZ + 0.00354776959069 WMT + 0.000921402141737 XOM>

In [13]:
p_risk

<gurobi.QuadExpr: 0.0 + [ 0.0014648350942 AA ^ 2 + 0.000576840280776 AXP * AA + 0.00044896026935 BA * AA + 0.00153240854887 BAC * AA + 0.00113116816965 CAT * AA + 0.000747468868915 CSCO * AA + 0.000507098648418 CVX * AA + 0.000740592695579 DD * AA + 0.000378077739299 DIS * AA + 0.000612087800424 GE * AA + 0.000427789307432 HD * AA + 0.00103593928395 HPQ * AA + 0.00044763842368 IBM * AA + 0.000496245812037 INTC * AA + 0.000241539038185 JNJ * AA + 0.000937812454134 JPM * AA + 0.000271607812829 KO * AA + 0.000242007922812 MCD * AA + 0.000461548877059 MMM * AA + 0.000277406508984 MRK * AA + 0.00054108364287 MSFT * AA + 0.000120879623331 PFE * AA + 0.000149432280798 PG * AA + 0.000269647420729 T * AA + 0.000471958158759 TRV * AA + 5.98688639272e-05 UNH * AA + 0.00058811896158 UTX * AA + 0.000196018531958 VZ * AA + 0.000126090139121 WMT * AA + 0.000486418268019 XOM * AA + 0.000576840280776 AA * AXP + 0.000713747061579 AXP ^ 2 + 0.000321301592267 BA * AXP + 0.00085931197724 BAC * AXP + 0.0004

To finish our base model, we will add an objective to minimize the risk, while keeping the total budget at 1.0. We'll set Gurobi's `Method` parameter to 1 to select the dual simplex method---we don't need to change this, but it helps ensure we arrive at a vertex solution with as few nonzeros as possible.

In [14]:
# Set the objective: minimize risk
m.setObjective(p_risk, GRB.MINIMIZE)

# Fix the budget
m.addConstr(p_total, GRB.EQUAL, 1)

# Select a simplex algorithm (to ensure a vertex solution)
m.setParam('Method', 1)

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1


### Sample models

We have set our objective to minimize risk, and fixed our budget at 1. The only thing we haven't done yet is set our target return. But let's solve the current model *as-is*, without that target. This will give us the *minimum risk* model.

In [15]:
m.optimize()

portfolio['Minimum risk'] = portvars.apply(lambda x:x.getAttr('x'))
portfolio

Optimize a model with 1 rows, 30 columns and 30 nonzeros
Model has 465 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 1 rows, 30 columns, 30 nonzeros
Presolved model has 465 quadratic objective terms

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s
      13    1.2988951e-04   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.05 seconds
Optimal objective  1.298895104e-04


,Variables,Minimum risk
AA,<gurobi.Var AA (value 0.0)>,0.000000
AXP,<gurobi.Var AXP (value 0.0)>,0.000000
BA,<gurobi.Var BA (value 0.0258534602102)>,0.025853
BAC,<gurobi.Var BAC (value 0.0)>,0.000000
CAT,<gurobi.Var CAT (value 0.0)>,0.000000
CSCO,<gurobi.Var CSCO (value 0.0)>,0.000000
CVX,<gurobi.Var CVX (value 0.0)>,0.000000
DD,<gurobi.Var DD (value 0.0)>,0.000000
DIS,<gurobi.Var DIS (value 0.0)>,0.000000
GE,<gurobi.Var GE (value 0.0)>,0.000000


For the next portfolio, we want a return that is 50% of the mean return obtained by the best-performing stock `BAC`, but with minimum risk.

In [16]:
# Add the return target
ret50 = 0.5 * extremes.loc['Mean_return','Maximum']
fixreturn = m.addConstr(p_return, GRB.EQUAL, ret50)

m.optimize()

portfolio['50% Max'] = portvars.apply(lambda x:x.getAttr('x'))
portfolio['Maximum'] = [1.0 if x == 'BAC' else 0.0 for x in syms]
portfolio

Optimize a model with 2 rows, 30 columns and 60 nonzeros
Model has 465 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-03, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2988951e-04   0.000000e+00   0.000000e+00      0s
      19    3.5277508e-04   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.02 seconds
Optimal objective  3.527750802e-04


,Variables,Minimum risk,50% Max,Maximum
AA,<gurobi.Var AA (value 0.0)>,0.000000,0.000000,0.0
AXP,<gurobi.Var AXP (value 0.0)>,0.000000,0.000000,0.0
BA,<gurobi.Var BA (value 0.0)>,0.025853,0.000000,0.0
BAC,<gurobi.Var BAC (value 0.141864595942)>,0.000000,0.141865,1.0
CAT,<gurobi.Var CAT (value 0.0)>,0.000000,0.000000,0.0
CSCO,<gurobi.Var CSCO (value 0.0)>,0.000000,0.000000,0.0
CVX,<gurobi.Var CVX (value 0.0)>,0.000000,0.000000,0.0
DD,<gurobi.Var DD (value 0.0)>,0.000000,0.000000,0.0
DIS,<gurobi.Var DIS (value 0.0)>,0.000000,0.000000,0.0
GE,<gurobi.Var GE (value 0.0)>,0.000000,0.000000,0.0


Time to plot the results. Of course, the results here are a bit deceptive, because we are plotting the performance of our optimal portfolios *over the very time period we have data*. So of course they will perform exactly as expected.

In [17]:
fig = figure(x_axis_type='datetime',y_axis_type='log',tools="pan,box_zoom,reset,resize")

pgrowth = data.dot(portfolio.loc[:,'Minimum risk':'Maximum'])
pgrowth = (pgrowth + 1).cumprod()

for (t,c) in zip(['Minimum risk','50% Max','Maximum'],['red','green','blue']):
    fig.line(pgrowth.index, pgrowth[t], legend=t, color=c)

fig.y_range = Range1d(1.0,1.1*pgrowth.max().max())
fig.legend.location="top_left"
show(fig)

### The efficient frontier

Now what we're going to do is sweep our return target over a range of values, starting at the smallest possible value (HPQ's dismal performance) to the largest (BAC's blowout). For each, we construct the minimum-risk portfolio. This will give us a tradeoff curve that is known in the business as the *efficient frontier* or the *Pareto-optimal curve*.

Note that we're using the *same model object* we've already constructed! All we have to do is set the return target and re-optimize for each value of interest.

In [18]:
# Turn off Gurobi's output
m.setParam('OutputFlag',False)

# Determine the range of returns. Make sure to include the lowest-risk
# portfolio in the list of options
minret = extremes.loc['Mean_return','Minimum']
maxret = extremes.loc['Mean_return','Maximum']
riskret = extremes.loc['Volatility','Minimizer']
riskret = stats.loc[riskret,'Mean_return']
returns = np.unique(np.hstack((np.linspace(minret,maxret,100),riskret)))

# Iterate through all returns
risks = returns.copy()
for k in range(len(returns)):
    fixreturn.rhs = returns[k]
    m.optimize()
    risks[k] = sqrt(p_risk.getValue())

Putting it all together on a single plot, we see the classic "bullet" shape of the Markowitz portfolio model. And we should not be surprised to see that the curve with shorting is far to the left of the curve without shorting. 

In [19]:
fig = figure(tools="pan,box_zoom,reset,resize")

# Individual stocks
fig.circle(stats['Volatility'], stats['Mean_return'], size=5, color='maroon')
fig.text(stats['Volatility'], stats['Mean_return'], syms, text_font_size='10px', x_offset=3, y_offset=-2)

# Divide the efficient frontier into two sections: those with
# a return less than the minimum risk portfolio, those that are greater.
tpos_n = returns >= riskret
tneg_n = returns <= riskret
fig.line(risks[tneg_n], returns[tneg_n], color='red')
fig.line(risks[tpos_n], returns[tpos_n], color='blue')

fig.xaxis.axis_label='Volatility (standard deviation)'
fig.yaxis.axis_label='Mean return'
fig.legend.orientation='bottom_left'
show(fig)